# Semantics construction 

In this notebook, we will combine GF and MMT to implement a Montague-style fragment of English.
We will use GF for parsing and MMT for describing the logic and semantics construction.

The example is intentionally very minimal.
Concretely, we want to support sentences like
* *John runs*
* *John runs and Mary runs*

## Grammar

The grammar is very simple. If you read the GF introduction notebook, you should be able to understand it easily.

In [1]:
abstract MiniGrammar = {
    cat
        S; NP; VP;
    fun
        pred : NP -> VP -> S;
        and : S -> S -> S;
        john : NP;
        mary : NP;
        run : VP;
}

Successfully imported MiniGrammar.gf

In [2]:
concrete MiniGrammarEng of MiniGrammar = {
    lincat
        S = Str; NP = Str; VP = Str;
    lin
        pred np vp = np ++ vp;
        and a b = a ++ "and" ++ b;
        john = "John";
        mary = "Mary";
        run = "runs";
}

Successfully imported MiniGrammarEng.gf

In [3]:
parse "John runs and Mary runs"

and (pred john run) (pred mary run)

In [4]:
parse "John runs and Mary runs" | vp -showfun

Dropdown(layout=Layout(width='max-content'), options=('0.0. and (pred john run) (pred mary run)',), value='0.0…

Image(value=b'<?xml version="1.0" encoding="UTF-8" sta...', format='svg+xml')

## Logic

The logic is very simple as well.
If you read the MMT intro, this should be very straight-forward:

In [5]:
theory minilogic : ur:?LF =
    prop : type ❘ # o ❙
    conjunction : o ⟶ o ⟶ o ❘ # 1 ∧ 2 prec 80 ❙
    // no need for other connectives ❙
    individual : type ❘ # ι ❙
❚

Successfully imported minilogic.mmt

We also need a **discourse domain theory (ddt)** that introduces all the constants we will need:

In [6]:
theory miniddt : ?minilogic =
    j : ι ❙       // john ❙
    m : ι ❙       // mary ❙
    r : ι ⟶ o ❙   // run (unary predicate) ❙
❚

Successfully imported miniddt.mmt

## Semantics construction

For the semantics construction need to represent abstract syntax trees in MMT.
For this we will make a **language theory** that corresponds to the abstract syntax:

In [7]:
theory minilanguage : ur:?LF =
    S : type ❙
    NP : type ❙
    VP : type ❙

    sentence : NP ⟶ VP ⟶ S ❙
    and : S ⟶ S ⟶ S ❙
    john : NP ❙
    mary : NP ❙
    run : VP ❙
❚

Successfully imported minilanguage.mmt

Creating a language theory from the abstract syntax is very straight-forward, so GLIF does it *automatically* for us in the background.
So this is the only time we will write it by hand.

We represent the semantics construction as an MMT **view**.
A few maps every symbol in a source theory to an expression in the target theory.
For example, we will map `S` (a type in the language theory) to propositions (`o`) as a sentence should be translated to a proposition.
Noun phrases (`NP`) will be mapped to individuals and verb phrases (`VP`) to unary predicates.

Afterwards, we will have to map all the function rules.
`john` will be mapped to `j`. Similarly, `mary` will be mapped to `m` and `run` to `r`.
`sentence` and `and` are a bit trickier and we will use lambda functions for that.
`and` should combine the arguments with a `∧`, i.e. we will map it to the function `[x, y] x ∧ y` and `sentence` should apply the verb phrase (a predicate) to the noun phrase (an individual).
Here is the full semantics construction:

In [8]:
view MiniGrammarSemantics : http://mathhub.info/tmpGLIF/default/MiniGrammar.gf?MiniGrammar -> ?miniddt =
    S = o ❙
    NP = ι ❙
    VP = ι⟶o ❙

    pred = [n:ι, v:ι⟶o] v n ❙
    and = [x, y] x ∧ y ❙
    john = j ❙
    mary = m ❙
    run = r ❙
❚


Successfully imported MiniGrammarSemantics.mmt

Now we can test it with the `construct` command.
By default, it uses the most recently imported view (i.e. the view from most recently run Jupyter cell).

In [9]:
parse "John runs and Mary runs" | construct

(r j)∧(r m)

To understand what is happening, we can also just view the AST and the result of the semantics construction without simplification (i.e. without β-reduction):

In [10]:
parse "John runs and Mary runs"
parse "John runs and Mary runs" | construct -no-simplify

and (pred john run) (pred mary run)

([x,y]x∧y) (([n:ι,v:ι⟶o]v n) j r) (([n:ι,v:ι⟶o]v n) m r)